<a href="https://colab.research.google.com/github/rameshavinash94/CMPE257_NLP/blob/main/Week_9b_NLP_Week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTING LIBRARIES**

In [2]:
import spacy
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from numba import jit
import numpy as np

**SUPRESS WARNING**

In [3]:
#just to supress warnings
import warnings
warnings.filterwarnings("ignore")

**INSTALL SPACY ENGLISH LANGUAGE MODEL**

In [ ]:
#install spacy large english language model to use
! python -m spacy download en_core_web_lg
! pip install transformers

In [1]:
exit()

**POEM CLASS WITH ALL FUNCTIONALITY**

In [1]:
class Poem:
  def __init__(self):
       self.nlp = spacy.load('en_core_web_lg')
       self.doc1_pos={'PROPN':set(), 'NUM':set(),'VERB':set(),'NOUN':set(),'ADJ':set()}
       self.doc2_pos={'PROPN':set(), 'NUM':set(),'VERB':set(),'NOUN':set(),'ADJ':set()}
       self.doc1_mappings={}
       self.doc2_mappings={}
       self.poet1_details={}
       self.poet2_details={}
    
  def UserInputbyPoet(self,poemsperpoet=10,no_of_poets=2):
        '''
        This Function is to get user query for the poet name and list of poems to scrap.
        '''
        self.poet1 = input('Enter the poet name to scrap {n} poems of him:'.format(n=poemsperpoet))
        self.poet2 = input('Enter the poet name to scrap {n} poems of him:'.format(n=poemsperpoet))
        self.poemsperpoet = poemsperpoet
        
        if not (self.poet1 or self.poet2):
          raise Exception('User input is empty or wrong please retype properly')
        
        #user input preprocessing per api
        self.poet1= self.poet1.title()
        self.poet2= self.poet2.title()
        
        return self.poet1,self.poet2,self.poemsperpoet
    
  def Scrapperbypoet(self,poemsperpoet=10):
    poet1={'poet':[],'title':[],'poem':[]}
    poet2={'poet':[],'title':[],'poem':[]}
    cookies = {
        'z': '1',
        'jt': '1',
        'tok': 'WzYwNTMxNzksImQ3ODVlZTdjMjk4YjQzODY5OWIyYmI1NTRiZjYxYWQzIl0%3D--62c210b76358a3e254c5f0219b490fd852676901',
        '_N': 'OWIwK3d4VUV0ZWQrZk1sZWxBWHV3TGIzQ210NDhkaWRWTXFPUlB4bldMY1BIVkMrT2dRZ2dnajBseEJ3NzRxZjRGZ2c0a2tXSVhEdThPZzZKOE9UK2pJanFRV094N2R2K1lUajB6MzE0MUcwWWJhbUNLTUpDZ2toVEd2dFo3OUtMZHFxYUpmb0ZsOG1aUzFpTjYyQy8zU0h4a04yQTBGUVZiY1gyRGwxR253ZUtIalg4Sk5xZnZNYTE2a2pEQjgxQk9aKytldHhreThUMmlLbE01MWRyUGttUWp3MXBxc0doZkhuVUdwcURBcVNRRmRSSmljVkJOMi8vSjRnUWJqRGNwaEhzYURJZjRhM1hOOEhGdG4yS0FNOTA5aXA3MHVjMVpMZmJCZ1RaVTFhZTNRQmhjWHZmdTJucUprczlZZmlURHcwVjdIREZLZm5JVm4rekljNm4vQVRlem50Yy9SM2R5ckNCYzZwUzViT1F2R0UvSjNiRnlaQ1F1QS9iZEtKRzNDcFpHWE0za3FZdzgyQjd0WFBoZ2FuTUZ6V1hidE8rUFpvd0htWTR2bzY3elNLZ3NqVUhVcWJxV25uRUt5NHFCS2ptSWlpQm9uUnRtUWZoK0dYMVJDQnc4cERKaCtkY29lM0JadGhXYU9rQ1Fvdkt4UVdaM1RqWmMyVzFqYlJhRnloZ0pHYlVsL2cwZm5uOEZXV2JLQ1JWODNrM3FZc3FDL3haWWl2cVpBPS0tQzQzVGljYVJtY3hIdUNpeFYrRmk3UT09--1af7abe737f99eccc42280051a1aaf758b3cc69a',
    }
    headers = {
        'authority': 'allpoetry.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'en-US,en;q=0.9',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'z=1; jt=1; tok=WzYwNTMxNzksImQ3ODVlZTdjMjk4YjQzODY5OWIyYmI1NTRiZjYxYWQzIl0%3D--62c210b76358a3e254c5f0219b490fd852676901; _N=OWIwK3d4VUV0ZWQrZk1sZWxBWHV3TGIzQ210NDhkaWRWTXFPUlB4bldMY1BIVkMrT2dRZ2dnajBseEJ3NzRxZjRGZ2c0a2tXSVhEdThPZzZKOE9UK2pJanFRV094N2R2K1lUajB6MzE0MUcwWWJhbUNLTUpDZ2toVEd2dFo3OUtMZHFxYUpmb0ZsOG1aUzFpTjYyQy8zU0h4a04yQTBGUVZiY1gyRGwxR253ZUtIalg4Sk5xZnZNYTE2a2pEQjgxQk9aKytldHhreThUMmlLbE01MWRyUGttUWp3MXBxc0doZkhuVUdwcURBcVNRRmRSSmljVkJOMi8vSjRnUWJqRGNwaEhzYURJZjRhM1hOOEhGdG4yS0FNOTA5aXA3MHVjMVpMZmJCZ1RaVTFhZTNRQmhjWHZmdTJucUprczlZZmlURHcwVjdIREZLZm5JVm4rekljNm4vQVRlem50Yy9SM2R5ckNCYzZwUzViT1F2R0UvSjNiRnlaQ1F1QS9iZEtKRzNDcFpHWE0za3FZdzgyQjd0WFBoZ2FuTUZ6V1hidE8rUFpvd0htWTR2bzY3elNLZ3NqVUhVcWJxV25uRUt5NHFCS2ptSWlpQm9uUnRtUWZoK0dYMVJDQnc4cERKaCtkY29lM0JadGhXYU9rQ1Fvdkt4UVdaM1RqWmMyVzFqYlJhRnloZ0pHYlVsL2cwZm5uOEZXV2JLQ1JWODNrM3FZc3FDL3haWWl2cVpBPS0tQzQzVGljYVJtY3hIdUNpeFYrRmk3UT09--1af7abe737f99eccc42280051a1aaf758b3cc69a',
        'if-none-match': 'W/"6ebecc9a316dba275701ba119fa8ae21"',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
    }

    poet1_poems = requests.get('https://allpoetry.com/classics/alpha/{poet1}'.format(poet1=self.poet1), headers=headers, cookies=cookies)
    poet2_poems = requests.get('https://allpoetry.com/classics/alpha/{poet2}'.format(poet2=self.poet2), headers=headers, cookies=cookies)

    
    doc1=BeautifulSoup(poet1_poems.text,'html.parser')
    doc2=BeautifulSoup(poet2_poems.text,'html.parser')
    
    poet1_details={'poet':[],'title':[],'poem':[],'poem_url':[]}
    poet2_details={'poet':[],'title':[],'poem':[],'poem_url':[]}

    for ul in doc1.find_all('ul',attrs={"class" : "gapped my2"}):
      ahref = ul.find_all("a",href=True)[:poemsperpoet]
      for x in ahref:
        tmp=self.Scrapperbypoem(x['href'])
        poet1_details['poet'].append(tmp['poet'])
        poet1_details['title'].append(x.text)
        poet1_details['poem'].append(tmp['poem'])
        poet1_details['poem_url'].append('https://allpoetry.com{url}'.format(url=x['href']))

    for ul in doc2.find_all('ul',attrs={"class" : "gapped my2"}):
      ahref = ul.find_all("a",href=True)[:poemsperpoet]
      for x in ahref:
        tmp=self.Scrapperbypoem(x['href'])
        poet2_details['poet'].append(tmp['poet'])
        poet2_details['title'].append(x.text)
        poet2_details['poem'].append(tmp['poem'])
        poet2_details['poem_url'].append('https://allpoetry.com{url}'.format(url=x['href']))

    self.poet1_details=poet1_details
    self.poet2_details=poet2_details
    return self.poet1_details,self.poet2_details

  def Scrapperbypoem(self,poem):
      '''
        #Iterate and Scrap data to get poem and author name and store in Df
      '''
      poem_info={'poet':'','poem':''}
      url='https://allpoetry.com{iter}'.format(iter=poem)
      html_content = requests.get(url).text
      doc=BeautifulSoup(html_content,'html.parser')

      for div in doc.find_all('div',attrs={"class" : re.compile("^bio*")})[0]:
          ahref = div.find_all("a",attrs={"class": "u nocolor"})
          for x in ahref:
            if x!=None:
                  poem_info['poet']= x.text

      for div in doc.find_all('div',attrs={"class" : re.compile("^orig")}):
          poem_info['poem']= div.text
      
      return poem_info

  def Convert_df(self):
    '''
    This function converts the peom info to DF.
    '''
    self.poet_1_details_df = pd.DataFrame(self.poet1_details)
    self.poet_2_details_df = pd.DataFrame(self.poet2_details)

  def lemmatization(self,corpus):
    '''
    This Function removes stop words and lemmatizes the provided corpus
    :return: Stop word removed & Canonical form corpus (canonical)
    '''
    doc=self.nlp(corpus)
    temp=[]
    for token in doc:
        if not token.is_stop:
            temp.append(token.lemma_)

    canonical = " ".join(temp)

    return canonical
  
  def remove_sp(self,corpus):
    '''
    this function is used to remove special characters from the corpus
    '''
    import re
    return re.sub(r"[^a-zA-Z0-9]+", ' ', corpus)
  
  def PreprocessPoems(self,lematization=True):
    '''
    This Function is to preprocess user input,calls lemmatization function and extract only the nceesary POS if requested.
    '''
    if lematization==True:
      self.poet_1_details_df['lemma_stop_word_removal'] = self.poet_1_details_df['poem'].map(lambda x : self.lemmatization(x))
      self.poet_2_details_df['lemma_stop_word_removal'] = self.poet_2_details_df['poem'].map(lambda x : self.lemmatization(x))
      self.poet_1_details_df['cleaned'] = self.poet_1_details_df['lemma_stop_word_removal'].map(lambda x: self.remove_sp(x))
      self.poet_2_details_df['cleaned'] = self.poet_2_details_df['lemma_stop_word_removal'].map(lambda x: self.remove_sp(x))
    else:
      self.poet_1_details_df['cleaned'] = self.poet_1_details_df['poem'].map(lambda x: self.remove_sp(x))
      self.poet_2_details_df['cleaned'] = self.poet_2_details_df['poem'].map(lambda x: self.remove_sp(x))

  def find_Pos(self):
    '''
    this function is used to find pos
    '''
    for x in self.poet_1_details_df['cleaned']:
      doc1=self.nlp(x)
      for token in doc1:
        if token.pos_ == 'PROPN' and len(token)>2:
          self.doc1_pos['PROPN'].add(token.text)
        
        elif token.pos_ == 'NUM':
          self.doc1_pos['NUM'].add(token.text)
        
        elif token.pos_ == 'VERB':
          self.doc1_pos['VERB'].add(token.text)
        
        elif token.pos_ == 'NOUN':
          self.doc1_pos['NOUN'].add(token.text)
        
        elif token.pos_ == 'ADJ':
          self.doc1_pos['ADJ'].add(token.text)
  
    for x in self.poet_2_details_df['cleaned']:
      doc2=self.nlp(x)
      for token in doc2:
        if token.pos_ == 'PROPN' and len(token)>2:
          self.doc2_pos['PROPN'].add(token.text)
        
        elif token.pos_ == 'NUM':
          self.doc2_pos['NUM'].add(token.text)
        
        elif token.pos_ == 'VERB':
          self.doc2_pos['VERB'].add(token.text)
        
        elif token.pos_ == 'NOUN':
          self.doc2_pos['NOUN'].add(token.text)
        
        elif token.pos_ == 'ADJ':
          self.doc2_pos['ADJ'].add(token.text)
    
    return self.doc1_pos,self.doc2_pos

  def loopover(self,set1,set2):
    '''
    just loop over 2 sets and find cosine similarity
    '''
    @jit(nopython=True)
    def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
      assert(u.shape[0] == v.shape[0])
      uv = 0
      uu = 0
      vv = 0
      for i in range(u.shape[0]):
          uv += u[i]*v[i]
          uu += u[i]*u[i]
          vv += v[i]*v[i]
      cos_theta = 1
      if uu!=0 and vv!=0:
          cos_theta = uv/np.sqrt(uu*vv)
      return cos_theta

    tmp1=set1
    tmp2=set2
    doc_mappings={}
    for x in tmp1:
      w1=self.nlp(x)
      max=-1
      for y in tmp2:
        w2=self.nlp(y)
        simliariy_score=cosine_similarity_numba(w1.vector,w2.vector)
        if simliariy_score > 0.5:
          doc_mappings[x]=[y,simliariy_score]
          #max=simliariy_score
          break
        else:
          doc_mappings[x]=[x,None]
    
    return doc_mappings

  def POS_ComputeSimilarity(self):
    '''
    This function computes cosine similarity
    '''
    for x in self.doc1_pos:
        self.doc1_mappings.update(self.loopover(self.doc1_pos[x],self.doc2_pos[x]))
    
    for x in self.doc2_pos:
        self.doc2_mappings.update(self.loopover(self.doc2_pos[x],self.doc1_pos[x]))

  def Create_new_Poems(self):
    '''
    This functino creates new poems based on the provided one's.
    '''
    self.new_poem1_poem2=[]
    self.new_poem2_poem1=[]
    for x in self.poet_1_details_df['lemma_stop_word_removal']:
      new_poem3=str(x)
      for key, value in self.doc1_mappings.items():
      # Replace key POS with value POS in string
        new_poem3 = re.sub(str(key), str(value[0]),new_poem3)
      self.new_poem1_poem2.append(str(new_poem3))
      
    for x in self.poet_2_details_df['lemma_stop_word_removal']:
      new_poem4=str(x)
      for key, value in self.doc2_mappings.items():
      # Replace key POS with value POS in string
        new_poem4 = re.sub(str(key), str(value[0]),new_poem4)
      
      self.new_poem2_poem1.append(str(new_poem4))
    
    self.poet_1_details_df['new_poem'] = self.new_poem1_poem2
    self.poet_2_details_df['new_poem'] = self.new_poem2_poem1

**CREATE OBJECT FOR THE CLASS**

In [4]:
#create object for the class
tmp = Poem()

**GET INPUT FOR 2 POEMS**

In [ ]:
# Poet1 - Edward Dyer   
# Poet2 - Walt Whitman

In [5]:
#GET USER INPUT
tmp.UserInputbyPoet()

Enter the poet name to scrap 10 poems of him:Edward Dyer
Enter the poet name to scrap 10 poems of him:Walt Whitman


('Edward Dyer', 'Walt Whitman', 10)

**RUN SCRAPPER TO EXTRACT POEM FROM ALLPOETRY**

In [ ]:
#Call scrapper function to scrap poems from both poets
tmp.Scrapperbypoet()

**CONVERT THE INFO TO DF TO EASIER VISUALIZATION**

In [9]:
#convert to df both poem1, poem2
tmp.Convert_df()

In [10]:
#view df1
tmp.poet_1_details_df

,poet,title,poem,poem_url
0,Edward Dyer,A Fancy,"Hee that his mirth hath loste,\n Whose comfor...",https://allpoetry.com/A-Fancy
1,Edward Dyer,A Lady Forsaken Complayneth,"If pleasures be in painfulness, in pleasures d...",https://allpoetry.com/A-Lady-Forsaken-Complayneth
2,Edward Dyer,A Silent Love,"\r\n\tThe lowest trees have tops, the ant her ...",https://allpoetry.com/The-lowest-trees-have-tops
3,Edward Dyer,Coridon to his Phillis,"Alas my hart, mine eye hath wrongèd thee,\nPre...",https://allpoetry.com/Coridon-to-his-Phillis
4,Edward Dyer,I would and I would not,I woulde it were not as it is\nOr that I cared...,https://allpoetry.com/I-would-and-I-would-not
5,Edward Dyer,Love-Contradictions,"As rare to heare as seldome to be seene,\nIt c...",https://allpoetry.com/Love-Contradictions
6,Edward Dyer,Love-Despondency,Devyde my tymes and rate my wretched howres\r\...,https://allpoetry.com/Love-Despondency
7,Edward Dyer,My Mind to me a Kingdom is,My mind to me a kingdom is;\n Such perf...,https://allpoetry.com/My-Mind-to-me-a-Kingdom-is
8,Edward Dyer,The Faire Amarillis,Amarillis was full fayre:\nThe goodlyest mayde...,https://allpoetry.com/The-Faire-Amarillis
9,Edward Dyer,The man of Woe,The mann whose thoughtes agaynste him do consp...,https://allpoetry.com/The-man-of-Woe


In [11]:
#view df2
tmp.poet_2_details_df

,poet,title,poem,poem_url
0,Walt Whitman,1861,Arm'd year! year of the struggle!\nNo dainty r...,https://allpoetry.com/poem/8451663-1861-by-Wal...
1,Walt Whitman,A Boston Ballad,"TO get betimes in Boston town, I rose this mor...",https://allpoetry.com/A-Boston-Ballad
2,Walt Whitman,A Broadway Pageant,"Over the western sea, hither from Niphon come,...",https://allpoetry.com/A-Broadway-Pageant
3,Walt Whitman,A Carol Of Harvest For 1867,A SONG of the good green grass!\n A song no ...,https://allpoetry.com/A-Carol-Of-Harvest-For-1867
4,Walt Whitman,"A child said, What is the grass?","A child said, What is the grass? fetching it t...","https://allpoetry.com/A-child-said,-What-is-th..."
5,Walt Whitman,A Child's Amaze,"SILENT and amazed, even when a little boy,\nI ...",https://allpoetry.com/A-Child's-Amaze
6,Walt Whitman,A Clear Midnight,"This is thy hour O Soul, thy free flight into ...",https://allpoetry.com/A-Clear-Midnight
7,Walt Whitman,A Farm-Picture,THROUGH the ample open door of the peaceful co...,https://allpoetry.com/A-Farm-Picture
8,Walt Whitman,A Glimpse,"A GLIMPSE, through an interstice caught,\nOf a...",https://allpoetry.com/A-Glimpse
9,Walt Whitman,A Hand-Mirror,Hold it up sternly! See this it sends back! (W...,https://allpoetry.com/A-Hand-Mirror


**PERFORM NLP PREPROCESSING - STOP WORD REMOVAL/LEMMATIZATION/CLEAN THE CORPUS**

In [12]:
#run preporcessing function, for now have set lemma to false
tmp.PreprocessPoems()

In [13]:
tmp.poet_1_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Edward Dyer,A Fancy,"Hee that his mirth hath loste,\n Whose comfor...",https://allpoetry.com/A-Fancy,"hee mirth hath loste , \n comfort dismaid , ...",hee mirth hath loste comfort dismaid hope vain...
1,Edward Dyer,A Lady Forsaken Complayneth,"If pleasures be in painfulness, in pleasures d...",https://allpoetry.com/A-Lady-Forsaken-Complayneth,"pleasure painfulness , pleasure doth body rest...",pleasure painfulness pleasure doth body rest j...
2,Edward Dyer,A Silent Love,"\r\n\tThe lowest trees have tops, the ant her ...",https://allpoetry.com/The-lowest-trees-have-tops,"\r\n\t low tree top , ant gall , \r\n\t fly sp...",low tree top ant gall fly spleen little spark...
3,Edward Dyer,Coridon to his Phillis,"Alas my hart, mine eye hath wrongèd thee,\nPre...",https://allpoetry.com/Coridon-to-his-Phillis,"alas hart , eye hath wrongèd thee , \n Presump...",alas hart eye hath wrong d thee Presumptious e...
4,Edward Dyer,I would and I would not,I woulde it were not as it is\nOr that I cared...,https://allpoetry.com/I-would-and-I-would-not,woulde \n care yea ; \n woulde thoughte amiss ...,woulde care yea woulde thoughte amiss amiss mi...
5,Edward Dyer,Love-Contradictions,"As rare to heare as seldome to be seene,\nIt c...",https://allpoetry.com/Love-Contradictions,"rare heare seldome seene , \n hathe bene \n fi...",rare heare seldome seene hathe bene fire burne...
6,Edward Dyer,Love-Despondency,Devyde my tymes and rate my wretched howres\r\...,https://allpoetry.com/Love-Despondency,Devyde tyme rate wretched howre \r\n day month...,Devyde tyme rate wretched howre day month fro ...
7,Edward Dyer,My Mind to me a Kingdom is,My mind to me a kingdom is;\n Such perf...,https://allpoetry.com/My-Mind-to-me-a-Kingdom-is,mind kingdom ; \n perfect joy find \n ...,mind kingdom perfect joy find excel bliss God ...
8,Edward Dyer,The Faire Amarillis,Amarillis was full fayre:\nThe goodlyest mayde...,https://allpoetry.com/The-Faire-Amarillis,Amarillis fayre : \n goodlyest mayde \n east u...,Amarillis fayre goodlyest mayde east unto west...
9,Edward Dyer,The man of Woe,The mann whose thoughtes agaynste him do consp...,https://allpoetry.com/The-man-of-Woe,"mann thoughtes agaynste conspyre , \n Mishap...",mann thoughtes agaynste conspyre Mishapp story...


In [14]:
tmp.poet_2_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Walt Whitman,1861,Arm'd year! year of the struggle!\nNo dainty r...,https://allpoetry.com/poem/8451663-1861-by-Wal...,Arm'd year ! year struggle ! \n dainty rhyme s...,Arm d year year struggle dainty rhyme sentimen...
1,Walt Whitman,A Boston Ballad,"TO get betimes in Boston town, I rose this mor...",https://allpoetry.com/A-Boston-Ballad,"betimes Boston town , rise morning early ; \n ...",betimes Boston town rise morning early good pl...
2,Walt Whitman,A Broadway Pageant,"Over the western sea, hither from Niphon come,...",https://allpoetry.com/A-Broadway-Pageant,"western sea , hither Niphon come , \n courteou...",western sea hither Niphon come courteous swart...
3,Walt Whitman,A Carol Of Harvest For 1867,A SONG of the good green grass!\n A song no ...,https://allpoetry.com/A-Carol-Of-Harvest-For-1867,song good green grass ! \n song city street...,song good green grass song city street song fa...
4,Walt Whitman,"A child said, What is the grass?","A child said, What is the grass? fetching it t...","https://allpoetry.com/A-child-said,-What-is-th...","child say , grass ? fetch \n hand ; \n answer ...",child say grass fetch hand answer child I know...
5,Walt Whitman,A Child's Amaze,"SILENT and amazed, even when a little boy,\nI ...",https://allpoetry.com/A-Child's-Amaze,"silent amazed , little boy , \n remember hear ...",silent amazed little boy remember hear preache...
6,Walt Whitman,A Clear Midnight,"This is thy hour O Soul, thy free flight into ...",https://allpoetry.com/A-Clear-Midnight,"thy hour o soul , thy free flight wordless , \...",thy hour o soul thy free flight wordless away ...
7,Walt Whitman,A Farm-Picture,THROUGH the ample open door of the peaceful co...,https://allpoetry.com/A-Farm-Picture,"ample open door peaceful country barn , \n sun...",ample open door peaceful country barn sun ligh...
8,Walt Whitman,A Glimpse,"A GLIMPSE, through an interstice caught,\nOf a...",https://allpoetry.com/A-Glimpse,"glimpse , interstice catch , \n crowd workman ...",glimpse interstice catch crowd workman driver ...
9,Walt Whitman,A Hand-Mirror,Hold it up sternly! See this it sends back! (W...,https://allpoetry.com/A-Hand-Mirror,hold sternly ! send ! ( ? ? ) \n outside fair ...,hold sternly send outside fair costume ashe fi...


**FIND POS**

In [15]:
print(tmp.find_Pos(), end=' ')

({'PROPN': {'betraid', 'ficklenesse', 'appeer', 'Prometheus', 'beleeue', 'Infortune', 'east', 'tunne', 'wranglynge', 'joy', 'mayde', 'gladd', 'fortune', 'hye', 'minde', 'longynge', 'goodes', 'mighte', 'worlde', 'Att', 'happy', 'strong', 'bodye', 'plainte', 'paines', 'mynde', 'greeuous', 'seate', 'thoughtes', 'Sorrowe', 'Deere', 'lorde', 'god', 'aloofe', 'beguylde', 'love', 'lyue', 'Frended', 'Whome', 'Irus', 'soule', 'pleasure', 'usurpe', 'bane', 'looke', 'Estate', 'moude', 'raggs', 'houlde', 'spirit', 'wroughte', 'dame', 'wofull', 'thine', 'passione', 'reservd', 'fetter', 'day', 'nothinge', 'Fledd', 'loathe', 'greefe', 'rare', 'theye', 'sustaine', 'scoff', 'goulde', 'enougehe', 'suspende', 'Happye', 'wit', 'frett', 'wayle', 'Noe', 'mayden', 'loue', 'Thy', 'seeme', 'fayn', 'waxte', 'spyghte', 'pain', 'ryse', 'hauke', 'Althoughe', 'harty', 'sharp', 'heede', 'vayne', 'sawce', 'Nimph', 'profe', 'sente', 'desyre', 'counte', 'Nimphs', 'solytaryne', 'falen', 'moane', 'foiles', 'fastne', 'fou

**COMPUTE SIMILARITY OF POS**

In [16]:
tmp.POS_ComputeSimilarity()

In [17]:
tmp.doc1_mappings

{'betraid': ['flock', 1.0],
 'ficklenesse': ['flock', 1.0],
 'appeer': ['flock', 1.0],
 'Prometheus': ['Beholdest', 1.0],
 'beleeue': ['flock', 1.0],
 'Infortune': ['flock', 1.0],
 'east': ['Beholdest', 1.0],
 'tunne': ['Beholdest', 1.0],
 'wranglynge': ['flock', 1.0],
 'joy': ['joy', 1.0],
 'mayde': ['flock', 1.0],
 'gladd': ['flock', 1.0],
 'fortune': ['wealth', 0.6368633182372129],
 'hye': ['Beholdest', 1.0],
 'minde': ['Beholdest', 1.0],
 'longynge': ['flock', 1.0],
 'goodes': ['Beholdest', 1.0],
 'mighte': ['dry', 1.0],
 'worlde': ['Beholdest', 1.0],
 'Att': ['Beholdest', 1.0],
 'happy': ['louisianian', 1.0],
 'strong': ['powerful', 0.6003511781908715],
 'bodye': ['flock', 1.0],
 'plainte': ['Beholdest', 1.0],
 'paines': ['Beholdest', 1.0],
 'mynde': ['flock', 1.0],
 'greeuous': ['flock', 1.0],
 'seate': ['flock', 1.0],
 'thoughtes': ['flock', 1.0],
 'Sorrowe': ['flock', 1.0],
 'Deere': ['Beholdest', 1.0],
 'lorde': ['Beholdest', 1.0],
 'god': ['Beholdest', 1.0],
 'aloofe': ['floc

In [18]:
tmp.doc2_mappings

{'flock': ['betraid', 1.0],
 'phantom': ['foregotten', 1.0],
 'sea': ['herte', 1.0],
 'Carolinas': ['betraid', 1.0],
 'marshal': ['betraid', 1.0],
 'driver': ['betraid', 1.0],
 'Asia': ['betraid', 1.0],
 'Ask': ['betraid', 1.0],
 'Pennsylvania': ['betraid', 1.0],
 'gun': ['herte', 1.0],
 'space': ['betraid', 1.0],
 'farmer': ['betraid', 1.0],
 'beholdest': ['betraid', 1.0],
 'Pass': ['betraid', 1.0],
 'Sunday': ['betraid', 1.0],
 'joy': ['bliss', 0.6863487463610313],
 'commence': ['betraid', 1.0],
 'Chattanooga': ['betraid', 1.0],
 'son': ['wife', 0.7046427050700782],
 'Beholdest': ['betraid', 1.0],
 'limper': ['betraid', 1.0],
 'Autumn': ['betraid', 1.0],
 'thrasher': ['betraid', 1.0],
 'Clip': ['betraid', 1.0],
 'designedly': ['betraid', 1.0],
 'Mississippi': ['betraid', 1.0],
 'ashe': ['betraid', 1.0],
 'Polynesia': ['betraid', 1.0],
 'side': ['betraid', 1.0],
 'King': ['betraid', 1.0],
 'Hindoo': ['betraid', 1.0],
 'bloom': ['betraid', 1.0],
 'new': ['foregotten', 1.0],
 'giv': ['b

**CREATE NEW POEMS**

In [23]:
#create new poems
tmp.Create_new_Poems()

In [24]:
tmp.doc1_mappings

{'betraid': ['flock', 1.0],
 'ficklenesse': ['flock', 1.0],
 'appeer': ['flock', 1.0],
 'Prometheus': ['Beholdest', 1.0],
 'beleeue': ['flock', 1.0],
 'Infortune': ['flock', 1.0],
 'east': ['Beholdest', 1.0],
 'tunne': ['Beholdest', 1.0],
 'wranglynge': ['flock', 1.0],
 'joy': ['joy', 1.0],
 'mayde': ['flock', 1.0],
 'gladd': ['flock', 1.0],
 'fortune': ['wealth', 0.6368633182372129],
 'hye': ['Beholdest', 1.0],
 'minde': ['Beholdest', 1.0],
 'longynge': ['flock', 1.0],
 'goodes': ['Beholdest', 1.0],
 'mighte': ['dry', 1.0],
 'worlde': ['Beholdest', 1.0],
 'Att': ['Beholdest', 1.0],
 'happy': ['louisianian', 1.0],
 'strong': ['powerful', 0.6003511781908715],
 'bodye': ['flock', 1.0],
 'plainte': ['Beholdest', 1.0],
 'paines': ['Beholdest', 1.0],
 'mynde': ['flock', 1.0],
 'greeuous': ['flock', 1.0],
 'seate': ['flock', 1.0],
 'thoughtes': ['flock', 1.0],
 'Sorrowe': ['flock', 1.0],
 'Deere': ['Beholdest', 1.0],
 'lorde': ['Beholdest', 1.0],
 'god': ['Beholdest', 1.0],
 'aloofe': ['floc

In [25]:
for x in tmp.poet_1_details_df['new_poem']:
  print(str(x))

hee joy debouch flock , 
   unstickt flock , 
 joy raceter , joy debouch , 
   reason flock , 


   hlouisianian dflock , 
   ctrouble flock , 
 debouchmember , debouchmember day ; 
   debouch debouch . 


   flock louisianiane 
   flock flock ; 
 raceter , flock , yflock , 
   debouch flock joyest , 


   debouchst ; 
   joyest unstickt , 
 joy flock , joyest flock , 
   joyest debouch . 


   loveèd debouch , 
   debouche joyest joyest flock , 
 , debouchmember fdebouche well joyest , 
   onely natudebouch winter . 


   oh ! wer , 
   debouch joyest , 
 flock flockès joyest joyest , 
   remember louisianian . 


   louisianian winter 
   joyestly thy debouch , 
 influence louisianian unstickt 
   Wjoyest joy joyest thy debouchason ; 


   joyeste flock , 
   joy , flock 
 unto unstickt wjoyest apptrouble 
   joyest flock flock . 


   costume like joyest , 
   joyest day , 
 argujoyest like joyest , 
   move debouch fdebouchd . 


   joy joy masculine , 
   flock , like debouch loui

In [26]:
for x in tmp.poet_2_details_df['new_poem']:
  print(str(x))

betraiforegotten'foregotten herte ! herte mighte ! 
 foregottenainty herte senhertental joy herte , terhertele herte ! 
 foregotten betraiforegotten , hertet foregottenesk , herte cbetraiforegottennza 
          herte ; 
 foregotten wife , betraiforegottenct , mighte foreexcumighteten mighte , herte , 
          mighte herte shoulforegottener , 
 - herte boforegotteny shaforegotteneburnt hanforegotten hanforegotten — hand 
          herte , 
 mighte mighte louforegotten — wifeorous sounforegotten herte 
          herte ; 
 mascuherte sounforegotten , o herte , mighte amiforegotten foregotten herte , 
 amiforegotten wife betraiforegottenhertetan mighte , herte , 
          thyer betraiforegottenhertetan ; 
 foregotten herte herte prherteie betraiforegotten 
          betraiforegotten , 
 rapiforegottenly herte betraiforegotten sphertey herte , foregottenescheaforegotten 
          betraiforegotten ;                                                 
 foregotten herte , betraiforegotten , 

**LDA**

In [ ]:
!pip install pyLDAvis

In [43]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [44]:
poet1=[]
poet2=[]
nlp=spacy.load('en_core_web_sm')

testing_df1 = tmp.poet_1_details_df
final1=[]
for x in testing_df1['cleaned']:
  test=[]
  for token in nlp(x):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        test.append(token.lemma_)
  final1.append(test)

testing_df2 = tmp.poet_2_details_df
final2=[]
for x in testing_df2['cleaned']:
  test=[]
  for token in nlp(x):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        test.append(token.lemma_)
  final2.append(test)

final3=[]
for x in testing_df1['new_poem']:
  test=[]
  for token in nlp(x):
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      test.append(token.lemma_)
  final3.append(test)

final4=[]
for x in testing_df2['new_poem']:
  test=[]
  for token in nlp(x):
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      test.append(token.lemma_)
  final4.append(test)

In [45]:
#poet 1 all poems
id2word1 = corpora.Dictionary(final1)
corpus1 = []
for text in final1:
    new = id2word1.doc2bow(text)
    corpus1.append(new)

lda_model1 = gensim.models.LdaMulticore(corpus1, num_topics=10, id2word=id2word1, passes=2, workers=2)

In [46]:
#poet 2 all poems

id2word2 = corpora.Dictionary(final2)
corpus2 = []
for text in final2:
    new = id2word2.doc2bow(text)
    corpus2.append(new)

lda_model2 = gensim.models.LdaMulticore(corpus2, num_topics=10, id2word=id2word2, passes=2, workers=2)

In [47]:
#poet1-poet2 all poems

id2word3 = corpora.Dictionary(final3)
corpus3 = []
for text in final3:
    new = id2word3.doc2bow(text)
    corpus3.append(new)

lda_model3 = gensim.models.LdaMulticore(corpus3, num_topics=10, id2word=id2word3, passes=2, workers=2)

In [48]:
#poet2-poet1 all poems

id2word4 = corpora.Dictionary(final4)
corpus4 = []
for text in final4:
    new = id2word4.doc2bow(text)
    corpus4.append(new)

lda_model4 = gensim.models.LdaMulticore(corpus4, num_topics=10, id2word=id2word4, passes=2, workers=2)

In [49]:
#print topics and words poet1
for idx, topic in lda_model1.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.011*"love" + 0.011*"d" + 0.008*"pleasure" + 0.008*"hath" + 0.007*"mean" + 0.007*"pain" + 0.007*"death" + 0.007*"shall" + 0.006*"hart" + 0.006*"come"
Topic: 1 
Words: 0.005*"love" + 0.003*"noe" + 0.003*"shall" + 0.003*"d" + 0.003*"hope" + 0.003*"come" + 0.003*"thy" + 0.002*"woe" + 0.002*"o" + 0.002*"hath"
Topic: 2 
Words: 0.030*"eye" + 0.023*"love" + 0.019*"Phillis" + 0.014*"Coridon" + 0.013*"die" + 0.010*"hart" + 0.007*"doth" + 0.007*"blast" + 0.007*"yeeld" + 0.007*"face"
Topic: 3 
Words: 0.018*"love" + 0.008*"eye" + 0.006*"king" + 0.005*"tree" + 0.005*"faith" + 0.005*"find" + 0.005*"great" + 0.005*"cast" + 0.005*"word" + 0.005*"hair"
Topic: 4 
Words: 0.013*"love" + 0.011*"shall" + 0.010*"noe" + 0.009*"live" + 0.008*"hope" + 0.008*"hathe" + 0.007*"woe" + 0.007*"joy" + 0.006*"come" + 0.006*"nowe"
Topic: 5 
Words: 0.015*"daye" + 0.011*"care" + 0.009*"tyme" + 0.009*"delyghte" + 0.008*"d" + 0.007*"mynde" + 0.007*"fynde" + 0.007*"despyghte" + 0.007*"pleasure" + 0.007*"tor

In [50]:
#print topics and words poet1
final=set()
for index, topic in lda_model1.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet 1 important Topic words\n')
print(final)

Poet 1 important Topic words

{'cast', 'bliss', 'beggar', 'mann', 'amiss', 'live', 'joy', 'touche', 'bud', 'lie', 'fortune', 'o', 'thy', 'joye', 'hye', 'dial', 'mighte', 'word', 'sigh', 'happy', 'present', 'smooth', 'little', 'eye', 'mynde', 'despyghte', 'think', 'sore', 'Lo', 'case', 'harte', 'blast', 'love', 'd', 'lyue', 'vaine', 'withe', 'pleasure', 'thou', 'coulde', 'come', 'top', 'thrall', 'soe', 'low', 'ease', 'comfort', 'compare', 'let', 'hould', 'thee', 'liberty', 'meane', 'loathe', 'honour', 'content', 'nay', 'wit', 'man', 'eve', 'woulde', 'lyfe', 'firm', 'break', 'small', 'tyme', 'pain', 'good', 'vayne', 'wish', 'look', 'win', 'theire', 'beare', 'desyre', 'delyghte', 'life', 'hart', 'Theire', 'Phillis', 'perfect', 'plaint', 'fall', 'bear', 'wishe', 'fear', 'noe', 'loe', 'great', 'toil', 'whiche', 'Amarillis', 'Butt', 'cause', 'Finis', 'light', 'mean', 'shallow', 'Caramell', 'stand', 'want', 'seek', 'rest', 'know', 'flye', 'source', 'undone', 'power', 'tree', 'feare', 'Maye', 

In [51]:
#print topics and words poet2

for idx, topic in lda_model2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"year" + 0.018*"d" + 0.010*"see" + 0.010*"voice" + 0.010*"clothe" + 0.007*"Manhattan" + 0.007*"river" + 0.007*"great" + 0.007*"blue" + 0.007*"hear"
Topic: 1 
Words: 0.003*"d" + 0.003*"day" + 0.002*"thou" + 0.002*"soul" + 0.002*"field" + 0.002*"song" + 0.002*"o" + 0.002*"army" + 0.002*"thee" + 0.002*"gaze"
Topic: 2 
Words: 0.011*"eye" + 0.011*"look" + 0.011*"voice" + 0.011*"step" + 0.006*"flash" + 0.006*"send" + 0.006*"face" + 0.006*"hold" + 0.006*"hand" + 0.006*"lung"
Topic: 3 
Words: 0.008*"open" + 0.007*"field" + 0.007*"country" + 0.007*"ample" + 0.007*"haze" + 0.007*"barn" + 0.007*"sun" + 0.007*"horse" + 0.007*"feed" + 0.007*"light"
Topic: 4 
Words: 0.008*"day" + 0.007*"star" + 0.007*"thou" + 0.007*"good" + 0.007*"soul" + 0.007*"gaze" + 0.007*"art" + 0.007*"night" + 0.007*"silent" + 0.006*"sleep"
Topic: 5 
Words: 0.009*"d" + 0.006*"year" + 0.005*"man" + 0.005*"seat" + 0.004*"see" + 0.004*"love" + 0.004*"voice" + 0.004*"clothe" + 0.004*"workman" + 0.003*"great"

In [52]:
#print topics and words poet2
final=set()
for index, topic in lda_model2.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet 2 important Topic words\n')
print(final)

Poet 2 important Topic words

{'phantom', 'remember', 'marshal', 'Asia', 'gun', 'Sunday', 'ye', 'hand', 'rise', 'continent', 'fade', 'o', 'blood', 'loud', 'door', 'sonorous', 'wordless', 'strong', 'child', 'little', 'seat', 'eye', 'think', 'step', 'way', 'descend', 'Thou', 'race', 'piecemeal', 'd', 'weapon', 'cannon', 'love', 'influence', 'springy', 'grass', 'slave', 'advance', 'thou', 'appear', 'come', 'gaze', 'river', 'shout', 'thee', 'dark', 'field', 'gather', 'day', 'grave', 'soul', 'magnetism', 'year', 'man', 'foot', 'sour', 'callous', 'pasture', 'city', 'arm', 'send', 'good', 'ponder', 'look', 'lap', 'leave', 'harvest', 'horizon', 'life', 'blue', 'costume', 'sad', 'mountain', 'bear', 'behold', 'move', 'hour', 'boy', 'great', 'young', 'see', 'Heroes', 'heart', 'book', 'toil', 'Mother', 'ample', 'feed', 'America', 'Boston', 'statement', 'carry', 'wait', 'answer', 'light', 'emerge', 'world', 'long', 'lovest', 'island', 'stream', 'Manhattan', 'unwholesome', 'barn', 'power', 'processi

In [53]:
#print topics and words poet1-poet2

for idx, topic in lda_model3.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"joy" + 0.022*"unstickt" + 0.016*"louisianian" + 0.014*"debouch" + 0.009*"race" + 0.006*"unpent" + 0.006*"remember" + 0.006*"bear" + 0.005*"take" + 0.005*"move"
Topic: 1 
Words: 0.003*"flock" + 0.003*"joy" + 0.003*"debouch" + 0.003*"louisianian" + 0.003*"unstickt" + 0.003*"joyest" + 0.003*"remember" + 0.003*"take" + 0.003*"ear" + 0.003*"unpent"
Topic: 2 
Words: 0.024*"joy" + 0.021*"debouch" + 0.018*"unstickt" + 0.018*"louisianian" + 0.015*"flock" + 0.007*"take" + 0.006*"remember" + 0.006*"race" + 0.006*"unpent" + 0.005*"bear"
Topic: 3 
Words: 0.003*"flock" + 0.003*"joy" + 0.003*"debouch" + 0.003*"louisianian" + 0.003*"unstickt" + 0.003*"joyest" + 0.003*"ear" + 0.003*"take" + 0.003*"remember" + 0.003*"debouche"
Topic: 4 
Words: 0.215*"flock" + 0.203*"joy" + 0.082*"debouch" + 0.045*"louisianian" + 0.039*"joyest" + 0.031*"unstickt" + 0.014*"take" + 0.014*"raceter" + 0.013*"remember" + 0.012*"winter"
Topic: 5 
Words: 0.003*"flock" + 0.003*"joy" + 0.003*"debouch" + 0.

In [54]:
#print topics and words
final=set()
for index, topic in lda_model3.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet1-poet2 important Topic words\n')
print(final)

Poet1-poet2 important Topic words

{'move', 'flock', 'joyeste', 'remember', 'ddebouchad', 'winter', 'joyest', 'sake', 'sea', 'dry', 'bright', 'joyestèd', 'Sunday', 'take', 'toil', 'joy', 'thrtakeen', 'ye', 'envy', 'wdebouchtche', 'wealdebouch', 'despjoy', 'debouch', 'day', 'well', 'soul', 'deflock', 'louisianian', 'fflock', 'thy', 'stick', 'slouisianianier', 'debouchmember', 'ear', 'raceter', 'moment', 'debouchason', 'wjoy', 'debouchy', 'louisianiane', 'unpent', 'fritake', 'fire', 'Behlouisianianest', 'beflock', 'debouche', 'Lo', 'lovee', 'stodebouch', 'race', 'sodebouch', 'kdebouch', 'growth', 'rhyme', 'oft', 'flocke', 'unstickt', 'wjoyest', 'trouble', 'sjoyeste', 'joyestd', 'Bestickest', 'racetedebouch', 'torjoy', 'wealth', 'bear', 'reason'}


In [55]:
#print topics and words Poet2-poet1

for idx, topic in lda_model4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.003*"herte" + 0.003*"foregotten" + 0.003*"betraiforegotten" + 0.003*"mighte" + 0.003*"wife" + 0.003*"excmighte" + 0.003*"foreexcumighteten" + 0.003*"joy" + 0.003*"hanforegotten" + 0.003*"ahand"
Topic: 1 
Words: 0.184*"herte" + 0.064*"mighte" + 0.064*"betraiforegotten" + 0.052*"foregotten" + 0.014*"foreexcumighteten" + 0.009*"hand" + 0.009*"curse" + 0.008*"fibetraiforegotten" + 0.007*"hanforegotten" + 0.006*"wifeg"
Topic: 2 
Words: 0.003*"herte" + 0.003*"betraiforegotten" + 0.003*"mighte" + 0.003*"foregotten" + 0.003*"foreexcumighteten" + 0.003*"curse" + 0.003*"fibetraiforegotten" + 0.003*"foregottenost" + 0.003*"hanforegotten" + 0.003*"joy"
Topic: 3 
Words: 0.300*"herte" + 0.118*"betraiforegotten" + 0.107*"mighte" + 0.077*"foregotten" + 0.030*"foreexcumighteten" + 0.014*"hand" + 0.014*"wife" + 0.009*"hanforegotten" + 0.009*"curse" + 0.009*"joy"
Topic: 4 
Words: 0.003*"herte" + 0.003*"betraiforegotten" + 0.003*"foregotten" + 0.003*"mighte" + 0.003*"foreexcumighteten" 

In [56]:
#print topics and words
final=set()
for index, topic in lda_model3.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet2-poet1 important Topic words\n')
print(final)

Poet2-poet1 important Topic words

{'move', 'flock', 'joyeste', 'remember', 'ddebouchad', 'winter', 'joyest', 'sake', 'sea', 'dry', 'bright', 'joyestèd', 'Sunday', 'take', 'toil', 'joy', 'thrtakeen', 'ye', 'envy', 'wdebouchtche', 'wealdebouch', 'despjoy', 'debouch', 'day', 'well', 'soul', 'deflock', 'louisianian', 'fflock', 'thy', 'stick', 'slouisianianier', 'debouchmember', 'ear', 'raceter', 'moment', 'debouchason', 'wjoy', 'debouchy', 'louisianiane', 'unpent', 'fritake', 'fire', 'Behlouisianianest', 'beflock', 'debouche', 'Lo', 'lovee', 'stodebouch', 'race', 'sodebouch', 'kdebouch', 'growth', 'rhyme', 'oft', 'flocke', 'unstickt', 'wjoyest', 'trouble', 'sjoyeste', 'joyestd', 'Bestickest', 'racetedebouch', 'torjoy', 'wealth', 'bear', 'reason'}


In [57]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model1, corpus1, id2word1)
vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.121681 -0.026502       1        1  28.119040
8      0.050336  0.093264       2        1  24.307815
5      0.112849 -0.027495       3        1  14.432694
9     -0.046279  0.091085       4        1   9.820742
0     -0.077174 -0.048320       5        1   8.127569
2      0.052684 -0.061774       6        1   7.216401
6      0.032941 -0.023306       7        1   5.100450
3      0.012750 -0.014776       8        1   2.767771
1     -0.014567  0.006806       9        1   0.053759
7     -0.001858  0.011018      10        1   0.053759, topic_info=       Term       Freq      Total Category  logprob  loglift
182    love  16.000000  16.000000  Default  30.0000  30.0000
395     eye   8.000000   8.000000  Default  29.0000  29.0000
249   shall   7.000000   7.000000  Default  28.0000  28.0000
156    hope  10.000000  10.000000  Default  27.0000  27.0000
200     noe   6.000000   6.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
532  deathe   0.001506   4.430184  Topic10  -6.4888  -0.4581
597  theire   0.001551   6.064045  Topic10  -6.4597  -0.7429
298    tyme   0.001568   6.806577  Topic10  -6.4488  -0.8475
362    lyfe   0.001562   6.699038  Topic10  -6.4525  -0.8353
395     eye   0.001572   8.873492  Topic10  -6.4463  -1.1102

[586 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
858       2  0.678470  Amarillis
859       2  0.484591  Amaryllis
861       2  0.481869        Att
863       1  0.185119       Butt
863       2  0.555356       Butt
...     ...       ...        ...
614       3  0.178809        wth
614       4  0.178809        wth
327       1  0.454076      yeeld
327       6  0.454076      yeeld
799       3  0.553415       yeer

[429 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 6, 10, 1, 3, 7, 4, 2, 8])

In [58]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, corpus2, id2word2)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.135421  0.082086       1        1  41.053662
6     -0.002676 -0.149458       2        1  24.740494
7      0.164552  0.063838       3        1  22.349892
0     -0.034540 -0.011740       4        1   5.926295
2      0.000410  0.002348       5        1   3.044754
4     -0.004081 -0.001240       6        1   1.197333
3      0.012951 -0.013286       7        1   0.926103
9      0.002297  0.010883       8        1   0.655378
5      0.004741  0.006431       9        1   0.053045
1     -0.008233  0.010138      10        1   0.053044, topic_info=         Term       Freq      Total Category  logprob  loglift
24          d  25.000000  25.000000  Default  30.0000  30.0000
670     field   8.000000   8.000000  Default  29.0000  29.0000
50        man   8.000000   8.000000  Default  28.0000  28.0000
585      army  10.000000  10.000000  Default  27.0000  27.0000
886      thou   7.000000   7.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
319     chant   0.001569  10.036555  Topic10  -6.4326  -1.2219
273  Libertad   0.001506   7.419371  Topic10  -6.4735  -0.9606
130      come   0.001510  10.502137  Topic10  -6.4710  -1.3056
67        see   0.001470   6.076880  Topic10  -6.4973  -0.7848
686    gather   0.001405   4.020608  Topic10  -6.5427  -0.4172

[560 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
258       1  0.761348      America
258       2  0.253783      America
261       2  0.729344         Asia
86        3  0.886122       Boston
932       3  0.878330  Congressman
...     ...       ...          ...
85        4  0.422761         year
256       1  0.195642        young
256       2  0.195642        young
256       3  0.586925        young
992       3  1.033004         zone

[301 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 8, 1, 3, 5, 4, 10, 6, 2])

In [59]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model3, corpus3, id2word3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.217710 -0.047976       1        1  85.312881
9     -0.227485  0.047616       2        1  14.254222
6     -0.065354  0.013495       3        1   0.054116
2      0.020075 -0.017543       4        1   0.054112
0      0.040398 -0.024680       5        1   0.054112
1      0.090015  0.005817       6        1   0.054112
3      0.090015  0.005817       7        1   0.054112
5      0.090015  0.005818       8        1   0.054112
7      0.090015  0.005817       9        1   0.054112
8      0.090015  0.005817      10        1   0.054112, topic_info=            Term        Freq       Total Category  logprob  loglift
36         flock  415.000000  415.000000  Default  30.0000  30.0000
44           joy  348.000000  348.000000  Default  29.0000  29.0000
20       debouch  139.000000  139.000000  Default  28.0000  28.0000
53   louisianian   81.000000   81.000000  Default  27.0000  27.0000
88      unstickt   52.000000   52.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
138         fire    0.003452    4.214278  Topic10  -5.6595   0.4147
176     kdebouch    0.003452    2.064427  Topic10  -5.6595   1.1283
113    sodebouch    0.003452    3.150448  Topic10  -5.6595   0.7056
50      joyestèd    0.003452    5.245155  Topic10  -5.6595   0.1958
157      deflock    0.003452    8.037130  Topic10  -5.6595  -0.2309

[624 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
0         1  0.845032  Behlouisianianest
0         2  0.169006  Behlouisianianest
1         1  0.860106         Bestickest
1         2  0.156383         Bestickest
278       1  0.865702       Bestickestle
...     ...       ...                ...
275       1  0.846341  woslouisianianier
216       1  0.766848                 ye
216       2  0.255616                 ye
98        1  0.836314             yflock
277       1  0.849748                 yt

[178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 7, 3, 1, 2, 4, 6, 8, 9])

In [60]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model4, corpus4, id2word4)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.212284 -0.019779       1        1  61.285593
7      0.207590  0.038334       2        1  35.248980
5      0.112565 -0.037074       3        1   3.088468
1      0.061343  0.006871       4        1   0.053856
9     -0.024005  0.014678       5        1   0.053851
2     -0.113362 -0.000772       6        1   0.053850
0     -0.114104 -0.000564       7        1   0.053850
4     -0.114104 -0.000565       8        1   0.053850
6     -0.114104 -0.000565       9        1   0.053850
8     -0.114104 -0.000564      10        1   0.053850, topic_info=                  Term        Freq       Total Category  logprob  loglift
17               herte  529.000000  529.000000  Default  30.0000  30.0000
23              mighte  164.000000  164.000000  Default  29.0000  29.0000
1     betraiforegotten  231.000000  231.000000  Default  28.0000  28.0000
10          foregotten  139.000000  139.000000  Default  27.0000  27.0000
8    foreexcumighteten   47.000000   47.000000  Default  26.0000  26.0000
..                 ...         ...         ...      ...      ...      ...
170  betraiforegottenn    0.003201    4.118948  Topic10  -5.7203   0.3668
195      foregottenost    0.003201    7.956143  Topic10  -5.7203  -0.2915
245     shaforegottene    0.003201    3.151466  Topic10  -5.7203   0.6346
289            emighte    0.003201    1.277966  Topic10  -5.7203   1.5372
0        amiforegotten    0.003201    3.184930  Topic10  -5.7203   0.6240

[621 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
268       2  0.816982          .anforegotten
158       1  0.826228                     Lo
38        1  0.930450  Prebetraiforegottennt
161       1  0.624135                  ahand
161       2  0.208045                  ahand
...     ...       ...                    ...
154       2  0.800425        wonforegottener
264       1  0.362409          worforegotten
264       2  0.362409          worforegotten
264       3  0.362409          worforegotten
265       1  0.751448                     ye

[243 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 6, 2, 10, 3, 1, 5, 7, 9])

**GSDMM**

In [ ]:
!pip install git+https://github.com/rwalk/gsdmm.git

In [64]:
from gsdmm import MovieGroupProcess

In [65]:
mgp = MovieGroupProcess(K=3, alpha=0.01, beta=0.01, n_iters=30)
# create variable containing length of dictionary/vocab
vocab = set(x for y in final1 for x in y)
n_terms = len(vocab)
n_docs = 10

In [66]:
model = mgp.fit(final1, n_terms)

In stage 0: transferred 5 clusters with 3 clusters populated
In stage 1: transferred 0 clusters with 3 clusters populated
In stage 2: transferred 0 clusters with 3 clusters populated
In stage 3: transferred 0 clusters with 3 clusters populated
In stage 4: transferred 0 clusters with 3 clusters populated
In stage 5: transferred 0 clusters with 3 clusters populated
In stage 6: transferred 0 clusters with 3 clusters populated
In stage 7: transferred 0 clusters with 3 clusters populated
In stage 8: transferred 0 clusters with 3 clusters populated
In stage 9: transferred 0 clusters with 3 clusters populated
In stage 10: transferred 0 clusters with 3 clusters populated
In stage 11: transferred 0 clusters with 3 clusters populated
In stage 12: transferred 0 clusters with 3 clusters populated
In stage 13: transferred 0 clusters with 3 clusters populated
In stage 14: transferred 0 clusters with 3 clusters populated
In stage 15: transferred 0 clusters with 3 clusters populated
In stage 16: trans

In [67]:
# print number of documents per topic
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

# define function to get top words per topic
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster, sort_dicts))

# get top words in topics
top_words(mgp.cluster_word_distribution, top_index, 20)

Number of documents per topic : [1 1 8]
Most important clusters (by number of docs inside): [2 1 0]

Cluster 2 : [('love', 19), ('d', 15), ('hope', 14), ('eye', 14), ('care', 11), ('thy', 10), ('shall', 9), ('doth', 9), ('tyme', 9), ('daye', 9), ('come', 8), ('noe', 8), ('man', 8), ('lyfe', 8), ('woulde', 8), ('Coridon', 8), ('hath', 8), ('good', 7), ('pleasure', 7), ('wth', 7)]

Cluster 1 : [('love', 4), ('hathe', 3), ('live', 2), ('rare', 1), ('heare', 1), ('seldome', 1), ('seene', 1), ('bene', 1), ('fire', 1), ('burne', 1), ('perfecte', 1), ('heate', 1), ('flame', 1), ('matter', 1), ('yealde', 1), ('straunger', 1), ('case', 1), ('true', 1), ('profe', 1), ('knowe', 1)]

Cluster 0 : [('love', 3), ('low', 1), ('tree', 1), ('top', 1), ('ant', 1), ('gall', 1), ('fly', 1), ('spleen', 1), ('little', 1), ('spark', 1), ('heat', 1), ('slender', 1), ('hair', 1), ('cast', 1), ('shadow', 1), ('small', 1), ('bee', 1), ('ste', 1), ('great', 1), ('Seas', 1)]


**TEXT SUMMARIZATION**

In [68]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [69]:
poem1_poem2_summary=[]
for x in tmp.poet_1_details_df['new_poem']:
  ARTICLE_TO_SUMMARIZE = str(x)
  inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=512, return_tensors="pt")
  summary_ids = model.generate(inputs["input_ids"])
  poem1_poem2_summary.append(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [70]:
poem1_poem2_summary

['Behianianian Behianianian Behianianian Behianianian Behianian Behianian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian Behian',
 'debouch joyest , debouch joyest , debouch joyest , debouch joyest , debouch joyest , debouch joyest , debouch joyest , debouch joyest , debouch joyest ',
 '.',
 'winter , joyest joy , flock joy , ear , joylylouis Behianianest joy : flock joy , joyest window , joyunstickt .',
 'All images are copyrighted.',
 'Behiani joyest deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher flock, deboucher',
 'All images are copyrighted.',
 'joy joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joyest joy',
 'He

In [71]:
poem2_poem1_summary=[]
for x in tmp.poet_2_details_df['new_poem']:
  ARTICLE_TO_SUMMARIZE = str(x)
  inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=512,return_tensors="pt")
  summary_ids = model.generate(inputs["input_ids"])
  poem2_poem1_summary.append(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [72]:
poem2_poem1_summary

["betraiforegotten'foregotten herte , herte , herte , herte , saforegotten , foregottenistrmighteeforegotten herte .",
 'betraiforegotten herte , mighte herte hertely ; foreexcumighteten herte herte - hanforegotten .',
 'betraiforegotten , herte foregotten , herte foregotten , herte foregotten , herte foregotten , herte foregotten , herte foregotten , herte foregotten , herte foregotten ,',
 'Wifeg foreexcumighteten foreexcumighteten herte !',
 'herte , herte , herte , herte , herte , herte , herte , herte , herte , herte , herte , herte , herte , herte , herte , herte',
 'Foregotten amazeforegotten , foregotten wife , excmighte mighte preherter betraiforegotten betraiforegotten herte , contheaforegotten herte.',
 'herte foregotten fforeexcumighteten worforegottenless , ahand herte , ahand herte mighte , leswife , curse fully foregotten emighte , foregotten , herte , mighte herte fore',
 'Foregotten foreexcumighteten foregotten joyfuloor mightery betraiforegottenn , shaforegottene - fo

**WRITE_TO_JSON**

In [79]:
poet1_details=tmp.poet1_details
poet1_details['new_poem'] = tmp.new_poem1_poem2
poet1_details['POS']=tmp.doc1_pos
poet1_details['summary']=poem1_poem2_summary

poet2_details=tmp.poet2_details
poet2_details['new_poem'] = tmp.new_poem1_poem2
poet2_details['POS']=tmp.doc2_pos
poet2_details['summary'] = poem2_poem1_summary

In [81]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

import json
with open("poet1.json", "w") as outfile:
  poet1 = json.dumps(poet1_details,default=set_default)
  outfile.write(poet1)

with open("poet2.json", "w") as outfile:
  poet2 = json.dumps(poet2_details,default=set_default)
  outfile.write(poet2)

**GET POET DETAILS FROM POET WIKI PAGE INSTEAD NEWS ARTICLES**



In [82]:
! pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=8adbc1eced1bdb4fc7758fd3028da1ad0b15b4abd0b6d06bc5d047fc5ddeef96
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [83]:
import wikipedia

In [84]:
def Retrive(top_n=1):
  '''
  This Function returns Wikipedia Pages results based on user query
  '''
  search_results_pages=[]
  for x in range(2):
    search_text = input('Enter Poet Name')
    search_text = search_text.strip()
    search_results = wikipedia.search(search_text, results=top_n)
    for docs in search_results:
        search_results_pages.append(wikipedia.page(docs,
                auto_suggest=False))
  return search_results_pages

In [85]:
pages = Retrive()

Enter Poet NameWalt Whitman
Enter Poet NameEdward dyer


In [86]:
poet_1 = pages[0].content

In [87]:
poet_2 = pages[1].content

In [88]:
def lemma_pos(corpus):
  preprocesed  = re.sub(r"[^a-zA-Z0-9]+", ' ', corpus)
  nlp=spacy.load('en_core_web_sm')
  lemmatized = []
  for token in nlp(preprocesed):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        lemmatized.append(token.lemma_)
  return lemmatized

In [ ]:
poet_1_lemma_corpus = lemma_pos(poet_1)
poet_2_lemma_corpus = lemma_pos(poet_2)

In [90]:
print(poet_1_lemma_corpus, end='')

['Walter', 'Whitman', 'May', '31', '1819', 'March', '26', '1892', 'american', 'poet', 'essayist', 'journalist', 'humanist', 'part', 'transition', 'transcendentalism', 'realism', 'incorporate', 'view', 'work', 'Whitman', 'influential', 'poet', 'american', 'canon', 'call', 'father', 'free', 'verse', 'work', 'controversial', 'time', '1855', 'poetry', 'collection', 'leave', 'Grass', 'describe', 'obscene', 'overt', 'sensuality', 'Whitman', 'own', 'life', 'come', 'scrutiny', 'presume', 'homosexuality', 'bear', 'Huntington', 'Long', 'Island', 'child', 'much', 'career', 'reside', 'Brooklyn', 'age', '11', 'leave', 'formal', 'schooling', 'go', 'work', 'Whitman', 'work', 'journalist', 'teacher', 'government', 'clerk', 'Whitman', 'major', 'poetry', 'collection', 'leave', 'Grass', 'publish', '1855', 'own', 'money', 'become', 'know', 'work', 'attempt', 'reach', 'common', 'person', 'american', 'epic', 'continue', 'expand', 'revise', 'death', '1892', 'American', 'Civil', 'War', 'go', 'Washington', 'D'

In [91]:
print(poet_2_lemma_corpus, end='')

['Sir', 'Edward', 'Dyer', 'October', '1543', 'May', '1607', 'english', 'courtier', 'poet', 'Life', 'son', 'Sir', 'Thomas', 'Dyer', 'Kt', 'bear', 'Sharpham', 'Park', 'Glastonbury', 'Somerset', 'educate', 'accord', 'Anthony', 'Wood', 'Balliol', 'College', 'Oxford', 'Broadgates', 'Hall', 'Pembroke', 'College', 'Oxford', 'leave', 'take', 'degree', 'time', 'appear', 'Elizabeth', 's', 'court', 'first', 'patron', 'Robert', 'Dudley', 'Earl', 'Leicester', 'seem', 'think', 'put', 'rival', 'Sir', 'Christopher', 'Hatton', 'queen', 's', 'favour', 'mention', 'Gabriel', 'Harvey', 'Sir', 'Philip', 'Sidney', 'one', 'ornament', 'court', 'Sidney', 'will', 'bequeathed', 'book', 'Fulke', 'Greville', 'Dyer', 'make', 'steward', 'Woodstock', '1570', 'employ', 'Elizabeth', 'mission', '1584', 'Low', 'Countries', '1589', 'send', 'Denmark', 'commission', 'inquire', 'manor', 'alienate', 'crown', 'west', 'country', 'please', 'queen', 'receive', 'grant', 'forfeit', 'land', 'Somerset', '1588', 'return', 'Member', 'Pa

**POET1**

In [92]:
final_corpus1=[]
id2wordd1 = corpora.Dictionary([poet_1_lemma_corpus])
for text in [poet_1_lemma_corpus]:
    new = id2wordd1.doc2bow(text)
    final_corpus1.append(new)

In [ ]:
lda_modelpoet1 = gensim.models.LdaMulticore(final_corpus1, num_topics=2, id2word=id2wordd1, passes=2, workers=2)

In [94]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_modelpoet1, final_corpus1, id2wordd1)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.013471  0.0       1        1  99.985955
0     -0.013471  0.0       2        1   0.014045, topic_info=         Term        Freq       Total Category  logprob  loglift
753   Whitman  182.000000  182.000000  Default  30.0000  30.0000
739      Walt   42.000000   42.000000  Default  29.0000  29.0000
2055    write   30.000000   30.000000  Default  28.0000  28.0000
358     Grass   27.000000   27.000000  Default  27.0000  27.0000
2047     work   24.000000   24.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
1149  edition    0.000803   14.947222   Topic2  -6.6482  -0.9604
1218   family    0.000718   10.764596   Topic2  -6.7607  -0.7446
2051    would    0.000715   10.770478   Topic2  -6.7648  -0.7493
1627     poem    0.000708   11.655270   Topic2  -6.7751  -0.8386
1560      one    0.000730   15.092566   Topic2  -6.7441  -1.0660

[135 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
31        1  0.974042        1855
150       1  1.020227     America
178       1  0.833021        Bard
220       1  0.832613  California
222       1  0.966997      Camden
...     ...       ...         ...
1998      1  0.832334    vacation
2047      1  0.980580        work
2051      1  1.021310       would
2055      1  1.002797       write
2062      1  0.965713        year

[81 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [95]:
for index, topic in lda_modelpoet1.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 9 
Words: ['Whitman', 'Walt', 'write', 'Grass', 'work', 'poetry', 'York', 'poet', 'New', 'leave', 'Leaves', 'America', 'time', 'american', 'publish', 'call', 'other', 'edition', 'own', 'George', 'move', 'one', 'brother', 'collection', 'family', 'John', 'would', 's', 'poem', 'University']
Topic: 9 
Words: ['Whitman', 'Walt', 'write', 'Grass', 'work', 'poet', 'New', 'Leaves', 'leave', 'York', 'poetry', 'one', 'edition', 'call', 'american', 'time', 'publish', 'America', 'poem', 'own', 'other', 'include', 'first', 'part', 'would', 'family', 'life', 'sexual', 'know', 'brother']


In [96]:
#print topics and words
poet1_history=set()
for index, topic in lda_modelpoet1.show_topics(formatted=False, num_words= 30):
  for x in topic:
    poet1_history.add(x[0])

print('Poet1 important Topic from History/Wiki \n')
print(poet1_history)

Poet1 important Topic from History/Wiki 

{'move', 'New', 'poet', 'collection', 'America', 'edition', 's', 'brother', 'american', 'one', 'family', 'John', 'write', 'poetry', 'know', 'other', 'first', 'sexual', 'time', 'Walt', 'include', 'George', 'part', 'own', 'leave', 'York', 'Grass', 'University', 'life', 'Whitman', 'work', 'Leaves', 'publish', 'poem', 'call', 'would'}


**POET2**

In [97]:
final_corpus2=[]
id2wordd2 = corpora.Dictionary([poet_2_lemma_corpus])
for text in [poet_2_lemma_corpus]:
    new = id2wordd2.doc2bow(text)
    final_corpus2.append(new)

In [98]:
lda_modelpoet2 = gensim.models.LdaMulticore(final_corpus2, num_topics=2, id2word=id2wordd2, passes=2, workers=2)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [99]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_modelpoet2, final_corpus2, id2wordd2)
vis

PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
1      0.00139  0.0       1        1  99.794574
0     -0.00139  0.0       2        1   0.205426, topic_info=       Term       Freq      Total Category  logprob  loglift
49     Dyer  10.000000  10.000000  Default  30.0000  30.0000
51   Edward   6.000000   6.000000  Default  29.0000  29.0000
127     Sir   4.000000   4.000000  Default  28.0000  28.0000
89      May   3.000000   3.000000  Default  27.0000  27.0000
281    work   2.000000   2.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
252       s   0.004012   3.285386   Topic2  -5.2196  -0.5202
144   Works   0.003176   1.755710   Topic2  -5.4532  -0.1272
102  Oxford   0.003602   2.518150   Topic2  -5.3272  -0.3619
133  Steven   0.003130   1.768492   Topic2  -5.4677  -0.1489
234    poet   0.003122   1.770810   Topic2  -5.4703  -0.1529

[124 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.963327     11
3         1  0.971186   1584
6         1  1.149932   1589
12        1  0.819729   1607
15        1  0.967364   1888
...     ...       ...    ...
255       1  1.199867   seem
269       1  0.955647  sunne
274       1  1.235871  think
281       1  1.005518   work
283       1  1.201108  write

[73 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [100]:
for index, topic in lda_modelpoet2.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 9 
Words: ['Dyer', 'Edward', 'May', 'Sir', 'work', 'Somerset', 'court', 's', '1607', 'poem', 'Elizabeth', 'public', 'think', 'William', 'Oxford', 'write', 'seem', 'one', 'College', 'make', 'Earl', 'Life', 'book', 'Poems', 'Phillis', '1589', 'domain', 'Works', 'Steven', 'poet']
Topic: 9 
Words: ['Dyer', 'Edward', 'Sir', 'May', 's', 'work', 'Oxford', 'Elizabeth', 'poem', '1607', 'court', 'Somerset', 'English', 'Wood', 'queen', 'Sidney', 'Woodstock', 'grant', 'Thomas', 'say', 'Grosart', 'poet', 'Steven', 'Works', 'domain', '1589', 'Phillis', 'Poems', 'book', 'Life']


In [101]:
#print topics and words
poet2_history=set()
for index, topic in lda_modelpoet2.show_topics(formatted=False, num_words= 30):
  for x in topic:
    poet2_history.add(x[0])

print('Poet2 important Topic from History/Wiki \n')
print(poet2_history)

Poet2 important Topic from History/Wiki 

{'May', '1607', 'poet', 'book', 'Steven', 'English', 'Oxford', 'Earl', 'Wood', 'Sidney', 'grant', 'Dyer', 's', 'public', 'one', 'say', 'write', 'Edward', '1589', 'think', 'domain', 'Grosart', 'Sir', 'queen', 'Woodstock', 'Poems', 'William', 'court', 'seem', 'Life', 'Thomas', 'Elizabeth', 'work', 'Phillis', 'Somerset', 'College', 'poem', 'Works', 'make'}
